In [1]:
library(Matching)
library(twang)
library(weights)
library(tidyverse)

Loading required package: MASS
## 
##  Matching (Version 4.9-2, Build Date: 2015-12-25)
##  See http://sekhon.berkeley.edu/matching for additional documentation.
##  Please cite software as:
##   Jasjeet S. Sekhon. 2011. ``Multivariate and Propensity Score Matching
##   Software with Automated Balance Optimization: The Matching package for R.''
##   Journal of Statistical Software, 42(7): 1-52. 
##

Loading required package: gbm
Loading required package: survival
Loading required package: lattice
Loading required package: splines
Loading required package: parallel
Loaded gbm 2.1.3
Loading required package: survey
Loading required package: grid
Loading required package: Matrix

Attaching package: ‘survey’

The following object is masked from ‘package:graphics’:

    dotchart

Loading required package: xtable
Loading required package: latticeExtra
Loading required package: RColorBrewer
Loading required package: Hmisc
Loading required package: Formula
Loading required package: ggplot2

At

In [2]:
data_dir <- file.path("..", "data")

In [3]:
full_data <- readRDS(file.path(data_dir, "full_data_ps.rds")) %>%
    mutate(sunday = factor(as.integer(icu_adm_weekday == 0), levels = c(0, 1)),
           monday = factor(as.integer(icu_adm_weekday == 1), levels = c(0, 1)),
           tuesday = factor(as.integer(icu_adm_weekday == 2), levels = c(0, 1)),
           wednesday = factor(as.integer(icu_adm_weekday == 3), levels = c(0, 1)),
           thursday = factor(as.integer(icu_adm_weekday == 4), levels = c(0, 1)),
           friday = factor(as.integer(icu_adm_weekday == 5), levels = c(0, 1)),
           saturday = factor(as.integer(icu_adm_weekday == 6), levels = c(0, 1))) %>%
    mutate(female = factor(as.integer(as.integer(as.character(gender)) == 0), levels = c(0, 1)))
head(full_data)

icustay_id,hadm_id,subject_id,first_careunit,intime,outtime,angus,age,icu_order,echo_time,⋯,ps,ps_weight,sunday,monday,tuesday,wednesday,thursday,friday,saturday,female
228416,159895,5491,0,2151-06-29 01:18:27,2151-07-27 16:00:56,1,76.58225,1,2151-06-30,⋯,0.5444486,1.836721,0,0,1,0,0,0,0,0
278148,169700,49395,0,2197-01-09 00:45:42,2197-01-10 06:46:29,1,82.72606,1,NA,⋯,0.2569337,1.345775,0,1,0,0,0,0,0,1
274174,103722,14855,0,2181-09-08 23:43:13,2181-09-26 16:52:02,1,61.84390,1,2181-09-11,⋯,0.3264142,3.063592,0,0,0,0,0,0,1,0
214954,170515,54642,0,2176-03-11 04:58:23,2176-03-15 18:00:48,1,63.86507,1,2176-03-11,⋯,0.8063102,1.240217,0,1,0,0,0,0,0,0
222457,183493,96815,0,2105-06-04 20:57:55,2105-06-06 20:01:51,1,62.84018,1,NA,⋯,0.2239520,1.288580,0,0,0,0,1,0,0,1
264413,134244,81436,0,2133-03-27 16:31:37,2133-04-04 16:15:28,1,62.60148,1,2133-03-31,⋯,0.3457691,2.892104,0,0,0,0,0,1,0,1


In [4]:
library(tableone)

In [5]:
tab_features <- CreateTableOne(vars = c("vs_map_first", "vs_heart_rate_first", "vs_temp_first", "vs_cvp_first",
                                        "lab_wbc_first", "lab_hemoglobin_first", "lab_platelet_first",
                                        "lab_sodium_first", "lab_potassium_first", "lab_bicarbonate_first",
                                        "lab_chloride_first", "lab_bun_first", "lab_lactate_first",
                                        "lab_creatinine_first", "lab_ph_first", "lab_po2_first", "lab_pco2_first",
                                        "lab_bnp_flag", "lab_troponin_flag", "lab_creatinine_kinase_flag"),
                               strata = "echo",
                               factorVars = c("lab_bnp_flag", "lab_troponin_flag", "lab_creatinine_kinase_flag"),
                               data = full_data)
tab_features$ContTable

                                   Stratified by echo
                                    0               1               p      test
  n                                 3099            3262                       
  vs_map_first (mean (sd))           79.91 (19.44)   80.03 (20.48)   0.810     
  vs_heart_rate_first (mean (sd))    93.01 (19.81)   95.07 (21.79)  <0.001     
  vs_temp_first (mean (sd))          36.75 (1.05)    36.85 (1.90)    0.006     
  vs_cvp_first (mean (sd))           11.88 (17.09)   13.74 (20.45)   0.024     
  lab_wbc_first (mean (sd))          13.48 (14.03)   13.76 (12.41)   0.401     
  lab_hemoglobin_first (mean (sd))   10.55 (1.98)    10.60 (2.04)    0.407     
  lab_platelet_first (mean (sd))    223.96 (134.79) 211.55 (128.46) <0.001     
  lab_sodium_first (mean (sd))      139.09 (6.45)   138.56 (5.89)    0.001     
  lab_potassium_first (mean (sd))     4.10 (0.79)     4.15 (0.84)    0.017     
  lab_bicarbonate_first (mean (sd))  22.28 (5.40)    22.35 (5.71) 

In [6]:
tab_basics <- CreateTableOne(vars = c("age", "gender", "weight", "elix_score",
                                      "saps", "sofa", "vent", "vaso", "sedative",
                                      "icd_chf", "icd_afib", "icd_renal", "icd_liver",
                                      "icd_copd", "icd_cad", "icd_stroke", "icd_malignancy",
                                      "icu_adm_weekday"),
                             strata = "echo",
                             factorVars = c("gender", "vent", "vaso", "sedative",
                                            "icd_chf", "icd_afib", "icd_renal", "icd_liver",
                                            "icd_copd", "icd_cad", "icd_stroke", "icd_malignancy",
                                            "icu_adm_weekday"),
                             data = full_data)

In [7]:
print(tab_basics, smd = TRUE)

                        Stratified by echo
                         0             1             p      test SMD   
  n                       3099          3262                           
  age (mean (sd))        66.69 (17.21) 65.74 (16.55)  0.025       0.056
  gender = 1 (%)          1524 (49.2)   1704 (52.2)   0.016       0.061
  weight (mean (sd))     78.56 (23.58) 83.17 (26.87) <0.001       0.182
  elix_score (mean (sd))  8.51 (7.45)  10.07 (7.67)  <0.001       0.207
  saps (mean (sd))       19.63 (5.79)  20.76 (5.44)  <0.001       0.202
  sofa (mean (sd))        5.31 (3.62)   6.33 (3.79)  <0.001       0.277
  vent = 1 (%)            1459 (47.1)   1901 (58.3)  <0.001       0.226
  vaso = 1 (%)             839 (27.1)   1228 (37.6)  <0.001       0.227
  sedative = 1 (%)        1247 (40.2)   1623 (49.8)  <0.001       0.192
  icd_chf = 1 (%)          563 (18.2)   1304 (40.0)  <0.001       0.495
  icd_afib = 1 (%)         622 (20.1)   1056 (32.4)  <0.001       0.282
  icd_renal = 1 (%)  

In [8]:
vars_ <- c("age", "female", "weight", "elix_score",
           "saps", "sofa", "vent", "vaso", "sedative",
           "icd_chf", "icd_afib", "icd_renal", "icd_liver",
           "icd_copd", "icd_cad", "icd_stroke", "icd_malignancy",
           "sunday", "monday", "tuesday", "wednesday",
           "thursday", "friday", "saturday",
           "vs_map_first", "vs_heart_rate_first", "vs_temp_first", "vs_cvp_first",
           "lab_wbc_first", "lab_hemoglobin_first", "lab_platelet_first",
           "lab_sodium_first", "lab_potassium_first", "lab_bicarbonate_first",
           "lab_chloride_first", "lab_bun_first", "lab_lactate_first",
           "lab_creatinine_first", "lab_ph_first", "lab_po2_first", "lab_pco2_first",
           "lab_bnp_flag", "lab_troponin_flag", "lab_creatinine_kinase_flag")

factor_vars_ <- c("gender", "vent", "vaso", "sedative",
                  "icd_chf", "icd_afib", "icd_renal", "icd_liver",
                  "icd_copd", "icd_cad", "icd_stroke", "icd_malignancy",
                  "sunday", "monday", "tuesday", "wednesday",
                  "thursday", "friday", "saturday",
                  "lab_bnp_flag", "lab_troponin_flag", "lab_creatinine_kinase_flag")

In [9]:
tabs <- CreateTableOne(vars = vars_, factorVars = factor_vars_,
                       strata = "echo", data = full_data,
                       argsNormal = list(var.equal = FALSE))

In [10]:
tabs_df <- print(tabs, smd = TRUE) %>% invisible %>% as.data.frame(stringsAsFactors = FALSE)

                                    Stratified by echo
                                     0               1               p     
  n                                    3099            3262                
  age (mean (sd))                     66.69 (17.21)   65.74 (16.55)   0.026
  female = 1 (%)                       1575 (50.8)     1558 (47.8)    0.016
  weight (mean (sd))                  78.56 (23.58)   83.17 (26.87)  <0.001
  elix_score (mean (sd))               8.51 (7.45)    10.07 (7.67)   <0.001
  saps (mean (sd))                    19.63 (5.79)    20.76 (5.44)   <0.001
  sofa (mean (sd))                     5.31 (3.62)     6.33 (3.79)   <0.001
  vent = 1 (%)                         1459 (47.1)     1901 (58.3)   <0.001
  vaso = 1 (%)                          839 (27.1)     1228 (37.6)   <0.001
  sedative = 1 (%)                     1247 (40.2)     1623 (49.8)   <0.001
  icd_chf = 1 (%)                       563 (18.2)     1304 (40.0)   <0.001
  icd_afib = 1 (%)               

In [11]:
tabs_df %>% head

,0,1,p,test,SMD
n,3099,3262,,,
age (mean (sd)),66.69 (17.21),65.74 (16.55),0.026,,0.056
female = 1 (%),1575 (50.8),1558 (47.8),0.016,,0.061
weight (mean (sd)),78.56 (23.58),83.17 (26.87),<0.001,,0.182
elix_score (mean (sd)),8.51 (7.45),10.07 (7.67),<0.001,,0.207
saps (mean (sd)),19.63 (5.79),20.76 (5.44),<0.001,,0.202


In [12]:
tabs_df_fmt <- tabs_df %>%
setNames(c("control", "treated", "pval", "test", "smd")) %>%
mutate(cov = rownames(.)) %>%
mutate(cov_name = str_split(cov, " ") %>% map_chr(first)) %>%
mutate(cate = cov_name %in% factor_vars_) %>%
mutate(ctrl = ifelse(cate,
                     str_extract(control, "\\((.*?)\\)") %>% str_replace_all("\\(|\\)", "") %>% sprintf("%s%%", .),
                     control)) %>%
mutate(trtd = ifelse(cate,
                     str_extract(treated, "\\((.*?)\\)") %>% str_replace_all("\\(|\\)", "") %>% sprintf("%s%%", .),
                     treated)) %>%
mutate(cov_fmt = str_replace_all(cov_name, "_", " ")) %>%
mutate(cov_fmt = str_replace_all(cov_fmt, "lab|vs|flag|icd|first", "")) %>%
mutate_all(funs(str_replace_all(., "\\s+$|^\\s+", ""))) %>%
mutate(cov_fmt = tools::toTitleCase(cov_fmt)) %>%
select(cov_fmt, ctrl, trtd, pval, smd) %>%
setNames(c("Covariate", "Non-Echo", "Echo", "p value", "SMD"))

In [13]:
tabs_df_fmt

Covariate,Non-Echo,Echo,p value,SMD
n,3099,3262,,
Age,66.69 (17.21),65.74 (16.55),0.026,0.056
Female,1575 (50.8),1558 (47.8),0.016,0.061
Weight,78.56 (23.58),83.17 (26.87),<0.001,0.182
Elix Score,8.51 (7.45),10.07 (7.67),<0.001,0.207
Saps,19.63 (5.79),20.76 (5.44),<0.001,0.202
Sofa,5.31 (3.62),6.33 (3.79),<0.001,0.277
Vent,47.1%,58.3%,<0.001,0.226
Vaso,27.1%,37.6%,<0.001,0.227
Sedative,40.2%,49.8%,<0.001,0.192


In [14]:
set.seed(4958)

In [15]:
ps_matches <- Match(Y = NULL, Tr = full_data$echo_int, X = full_data$ps, M = 1,
                    estimand = "ATT", caliper = 0.01,
                    exact = FALSE, replace = FALSE)

Warning message in Match(Y = NULL, Tr = full_data$echo_int, X = full_data$ps, M = 1, :
“replace==FALSE, but there are more (weighted) treated obs than control obs.  Some treated obs will not be matched.  You may want to estimate ATC instead.”

In [16]:
tab <- table(full_data$mort_28_day[ps_matches$index.treated],
             full_data$mort_28_day[ps_matches$index.control],
             dnn = c("Echo", "Control"))
tab

    Control
Echo   0   1
   0 846 383
   1 279 121

In [17]:
tabs_ps <- CreateTableOne(vars = vars_, factorVars = factor_vars_,
                          strata = "echo", argsNormal = list(var.equal = FALSE),
                          data = full_data[unlist(ps_matches[c("index.treated", "index.control")]), ])

In [18]:
tabs_ps_df <- print(tabs_ps, smd = TRUE) %>% as.data.frame(stringsAsFactors = FALSE)

                                    Stratified by echo
                                     0               1               p     
  n                                    1629            1629                
  age (mean (sd))                     66.63 (16.51)   65.85 (16.95)   0.180
  female = 1 (%)                        776 (47.6)      791 (48.6)    0.623
  weight (mean (sd))                  81.50 (24.98)   79.27 (22.67)   0.011
  elix_score (mean (sd))               9.57 (7.69)     8.95 (7.63)    0.021
  saps (mean (sd))                    20.35 (5.41)    19.75 (5.14)    0.001
  sofa (mean (sd))                     6.00 (3.75)     5.53 (3.47)   <0.001
  vent = 1 (%)                          880 (54.0)      804 (49.4)    0.009
  vaso = 1 (%)                          524 (32.2)      487 (29.9)    0.173
  sedative = 1 (%)                      740 (45.4)      688 (42.2)    0.072
  icd_chf = 1 (%)                       460 (28.2)      350 (21.5)   <0.001
  icd_afib = 1 (%)               

In [19]:
tabs_ps_df %>% head

,0,1,p,test,SMD
n,1629,1629,,,
age (mean (sd)),66.63 (16.51),65.85 (16.95),0.180,,0.047
female = 1 (%),776 (47.6),791 (48.6),0.623,,0.018
weight (mean (sd)),81.50 (24.98),79.27 (22.67),0.011,,0.093
elix_score (mean (sd)),9.57 (7.69),8.95 (7.63),0.021,,0.081
saps (mean (sd)),20.35 (5.41),19.75 (5.14),0.001,,0.115


In [20]:
tabs_ps_df_fmt <- tabs_ps_df %>%
setNames(c("control", "treated", "pval", "test", "smd")) %>%
mutate(cov = rownames(.)) %>%
mutate(cov_name = str_split(cov, " ") %>% map_chr(first)) %>%
mutate(cate = cov_name %in% factor_vars_) %>%
mutate(ctrl = ifelse(cate,
                     str_extract(control, "\\((.*?)\\)") %>% str_replace_all("\\(|\\)", "") %>% sprintf("%s%%", .),
                     control)) %>%
mutate(trtd = ifelse(cate,
                     str_extract(treated, "\\((.*?)\\)") %>% str_replace_all("\\(|\\)", "") %>% sprintf("%s%%", .),
                     treated)) %>%
mutate(cov_fmt = str_replace_all(cov_name, "_", " ")) %>%
mutate(cov_fmt = str_replace_all(cov_fmt, "lab|vs|flag|icd|first", "")) %>%
mutate_all(funs(str_replace_all(., "\\s+$|^\\s+", ""))) %>%
mutate(cov_fmt = tools::toTitleCase(cov_fmt)) %>%
select(cov_fmt, ctrl, trtd, pval, smd) %>%
setNames(c("Covariate", "Non-Echo (PS matching)", "Echo (PS matching)", "p value (PS matching)", "SMD (PS matching)"))

In [21]:
tabs_ps_df_fmt

Covariate,Non-Echo (PS matching),Echo (PS matching),p value (PS matching),SMD (PS matching)
n,1629,1629,,
Age,66.63 (16.51),65.85 (16.95),0.180,0.047
Female,776 (47.6),791 (48.6),0.623,0.018
Weight,81.50 (24.98),79.27 (22.67),0.011,0.093
Elix Score,9.57 (7.69),8.95 (7.63),0.021,0.081
Saps,20.35 (5.41),19.75 (5.14),0.001,0.115
Sofa,6.00 (3.75),5.53 (3.47),<0.001,0.129
Vent,54.0%,49.4%,0.009,0.093
Vaso,32.2%,29.9%,0.173,0.049
Sedative,45.4%,42.2%,0.072,0.064


In [22]:
data.table::fwrite(tabs_df_fmt, file.path(data_dir, "tableone_smd.csv"))
data.table::fwrite(tabs_ps_df_fmt, file.path(data_dir, "tableone_ps_smd.csv"))

In [23]:
tbone <- data.table::fread(file.path(data_dir, "tableone.csv"), data.table = FALSE)
tbone

Covariate,Echo,Non-Echo,p value,Echo (weighted cohort),Non-Echo (weighted cohort),p value (weighted cohort)
Age,65.74 (16.55),66.69 (17.21),0.026,65.99 (16.62),66.82 (16.75),0.048
Female,47.76%,50.82%,0.016,48.29%,49.27%,0.434
Weight,83.17 (26.87),78.56 (23.58),<0.001,81.36 (25.39),80.38 (24.35),0.135
Elix Score,10.07 (7.67),8.51 (7.45),<0.001,9.48 (7.63),9.14 (7.55),0.072
Saps,20.76 (5.44),19.63 (5.79),<0.001,20.25 (5.42),20.02 (5.64),0.103
Sofa,6.33 (3.79),5.31 (3.62),<0.001,5.89 (3.70),5.68 (3.67),0.025
Vent,58.28%,47.08%,<0.001,53.63%,50.61%,0.016
Vaso,37.65%,27.07%,<0.001,33.64%,30.55%,0.008
Sedative,49.75%,40.24%,<0.001,45.83%,42.92%,0.019
Chf,39.98%,18.17%,<0.001,31.10%,26.09%,<0.001


In [24]:
tabs_full <- tabs_df_fmt %>% select(Covariate, SMD) %>%
full_join(tbone, by = "Covariate") %>%
full_join(tabs_ps_df_fmt, by = "Covariate")

tabs_full

Covariate,SMD,Echo,Non-Echo,p value,Echo (weighted cohort),Non-Echo (weighted cohort),p value (weighted cohort),Non-Echo (PS matching),Echo (PS matching),p value (PS matching),SMD (PS matching)
n,,NA,NA,NA,NA,NA,NA,1629,1629,,
Age,0.056,65.74 (16.55),66.69 (17.21),0.026,65.99 (16.62),66.82 (16.75),0.048,66.63 (16.51),65.85 (16.95),0.180,0.047
Female,0.061,47.76%,50.82%,0.016,48.29%,49.27%,0.434,776 (47.6),791 (48.6),0.623,0.018
Weight,0.182,83.17 (26.87),78.56 (23.58),<0.001,81.36 (25.39),80.38 (24.35),0.135,81.50 (24.98),79.27 (22.67),0.011,0.093
Elix Score,0.207,10.07 (7.67),8.51 (7.45),<0.001,9.48 (7.63),9.14 (7.55),0.072,9.57 (7.69),8.95 (7.63),0.021,0.081
Saps,0.202,20.76 (5.44),19.63 (5.79),<0.001,20.25 (5.42),20.02 (5.64),0.103,20.35 (5.41),19.75 (5.14),0.001,0.115
Sofa,0.277,6.33 (3.79),5.31 (3.62),<0.001,5.89 (3.70),5.68 (3.67),0.025,6.00 (3.75),5.53 (3.47),<0.001,0.129
Vent,0.226,58.28%,47.08%,<0.001,53.63%,50.61%,0.016,54.0%,49.4%,0.009,0.093
Vaso,0.227,37.65%,27.07%,<0.001,33.64%,30.55%,0.008,32.2%,29.9%,0.173,0.049
Sedative,0.192,49.75%,40.24%,<0.001,45.83%,42.92%,0.019,45.4%,42.2%,0.072,0.064


In [25]:
data.table::fwrite(tabs_full, file.path(data_dir, "tableone_full.csv"))